In [1]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [2]:
print(text)


メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。



In [3]:
import MeCab

In [4]:
tagger = MeCab.Tagger("")
lines = tagger.parse(text).splitlines()


In [5]:
# 30
verbs = []

for line in lines:
    if line == "EOS" or line == "":
        continue
    parts = line.split("\t")
    surface = parts[0]
    feature = parts[4]
    features = feature.split("-")
    
    if features[0] == "動詞":
        verbs.append(surface)

print(verbs)


['し', '除か', 'なら', 'し', 'わから', 'ある', '吹き', '遊ん', '暮し', '来', '対し', 'あっ']


In [6]:
# 31
base_verbs = []

for line in lines:
    if line == "EOS" or line == "":
        continue
    parts = line.split("\t")
    base = parts[3]
    feature = parts[4]
    features = feature.split("-")
    
    if features[0] == "動詞":
        base_verbs.append(base)

print(base_verbs)


['為る', '除く', '成る', '為る', '分かる', '有る', '吹く', '遊ぶ', '暮らす', '来る', '対する', '有る']


In [7]:
# 32
tokens = []
for line in tagger.parse(text).splitlines():
    if line == "EOS" or not line.strip():
        continue
    parts = line.split("\t")
    surface = parts[0]
    feature = parts[4]
    features = feature.split("-")
    tokens.append((surface, features[0]))

results = []
for i in range(len(tokens) - 2):
    if tokens[i][1] == "名詞" and tokens[i + 1][0] == "の" and tokens[i + 2][1] == "名詞":
        results.append(tokens[i][0] + "の" + tokens[i + 2][0])

print(results)


['暴虐の王', '村の牧人']
